In [1]:
%load_ext autoreload
%autoreload 2

import os, sys, shutil, bz2, copy
from pathlib import Path
import pandas as pd
pd.set_option('display.max_columns', 50)
import numpy as np

### Plotting imports ###
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.colors as mcolors
import matplotlib as mpl
from matplotlib.patches import StepPatch
import matplotlib.ticker as ticker
import matplotlib.gridspec as gridspec
import logomaker as lm
palette = list(mcolors.TABLEAU_COLORS.keys())
sns.set_theme(style="ticks", palette="muted")
sns.set_context("talk")
%matplotlib inline

/home/sulab/anaconda3/lib/python3.9/site-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/sulab/anaconda3/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/sulab/anaconda3/lib/python3.9/site-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/sulab/anaconda3/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)


In [2]:
# Navigate back to NBdir in case of re-running a code block:
if not 'NBdir' in globals():
    NBdir = os.getcwd()
print('Notebook is in: {}'.format(NBdir))
os.chdir(NBdir)  # If you changed the current working dir, this will take you back to the notebook dir.

# Define the path to the repo folder.
# Change if necessary.
homedir = '/'.join(NBdir.split('/')[0:-2])
print('Repo is in: {}'.format(homedir))
sys.path.insert(1, homedir)
from src.misc import index_to_sample_df, downsample_raw_input, read_tRNAdb_info, sample_df_to_dict
from src.read_processing import AR_merge, BC_split, Kmer_analysis, BC_analysis, UMI_trim
from src.alignment import SWIPE_align
from src.stats_collection import STATS_collection
from src.plotting import TRNA_plot
from src.transcript_mutations import TM_analysis

# These are default folder names for data and raw fastq files
# relative to the folder in which this notebook is in:
data_dir = 'data'
seq_dir = 'raw_fastq'
seq_dir_noDS = seq_dir # Not downsampled

# These folder names are used in subsequent processing steps
# to dump data. Best to not change:
AdapterRemoval_dir = 'AdapterRemoval'
BC_dir = 'BC_split'
UMI_dir = 'UMI_trimmed'
align_dir = 'SWalign'
stats_dir = 'stats_collection'
TM_dir = 'transcript_mutations'
plotting_dir = 'plotting'
tRNA_database = dict()
tRNA_database['human'] = '{}/tRNA_database/human/hg38-tRNAs.fa'.format(homedir)
tRNA_database['mouse'] = '{}/tRNA_database/mouse/mm10-tRNAs.fa'.format(homedir)
tRNA_database_masked = dict()
tRNA_database_masked['human'] = '{}/tRNA_database_masked/human/human-tRNAs.fa'.format(homedir)
# Read information (length, codon etc) of tRNAs into dictionary:
tRNA_data = read_tRNAdb_info(tRNA_database)
SWIPE_score_mat = '{}/utils/nuc_score-matrix.txt'.format(homedir)
SWIPE_score_mat2 = '{}/utils/nuc_score-matrix_2.txt'.format(homedir) # For masked reference sequences
# tRNA sequencing yields many duplicated reads.
# Adding these commonly seen sequences to a list prevents duplicated alignment:
common_seqs = '{}/utils/common-seqs.fasta.bz2'.format(homedir)

# Define minimum read length based on minimum insert size:
MIN_INSERT_LEN = 10
UMI_LEN = 10
BC_MAX_LEN = 19
MIN_READ_LEN = MIN_INSERT_LEN + UMI_LEN + BC_MAX_LEN
print('Using minimum read length: {} (after merge)'.format(MIN_READ_LEN))

# Read index information:
index_list_fnam = 'index_list.xlsx'
index_df = pd.read_excel('{}/utils/{}'.format(homedir, index_list_fnam))

Notebook is in: /home/sulab/tRNA-charge-seq/projects/RT-comp
Repo is in: /home/sulab/tRNA-charge-seq
Using minimum read length: 39 (after merge)


In [3]:
### Input settings ###
sample_list_fnam = 'sample_list.xlsx'
sample_df = pd.read_excel('{}/{}'.format(NBdir, sample_list_fnam))
# Add barcode sequences:
sample_df = index_to_sample_df(sample_df, index_df)
# Read elementary info (replicate, barcode, species)
# for each unique sample name into a dictionary:
sample_dict = sample_df_to_dict(sample_df)
# Get filenames from the sample information:
inp_file_df = sample_df[['fastq_mate1_filename', 'fastq_mate2_filename', 'P5_index', 'P7_index', 'P5_index_seq', 'P7_index_seq']].copy().drop_duplicates().reset_index(drop=True)

# Downsample:
if False:
    sample_df, inp_file_df, seq_dir = downsample_raw_input(sample_df, inp_file_df, NBdir, data_dir, seq_dir_noDS, downsample_absolute=1e4)

# Make a dictionary with paths used for data processing:
dir_dict = dict(NBdir = NBdir,
                data_dir = data_dir,
                seq_dir = seq_dir,
                AdapterRemoval_dir = AdapterRemoval_dir,
                BC_dir = BC_dir,
                UMI_dir = UMI_dir,
                align_dir = align_dir,
                stats_dir = stats_dir,
                TM_dir = TM_dir,
                plotting_dir = plotting_dir)

In [4]:
### Run AdapterRemoval ###
AR_obj = AR_merge(dir_dict, inp_file_df, MIN_READ_LEN, overwrite_dir=True, \
                  AR_threads=6)
inp_file_df = AR_obj.run_parallel(n_jobs=2, overwrite=True)

In [5]:
### Split files based on barcodes ###
BCsplit_obj = BC_split(dir_dict, sample_df, inp_file_df, overwrite_dir=True)
sample_df, inp_file_df = BCsplit_obj.run_parallel(n_jobs=12, load_previous=False)

In [6]:
### Generate UMI stats and write final trimmed tRNA sequences ###
UMItrim_obj = UMI_trim(dir_dict, sample_df, overwrite_dir=True)
sample_df = UMItrim_obj.run_parallel(n_jobs=6, load_previous=False)
sample_df.head(3)

Downsampling UMI trimmed sequences to maximum 2000000 reads.


,sample_name_unique,sample_name,replicate,fastq_mate1_filename,fastq_mate2_filename,P5_index,P7_index,barcode,species,plot_group,hue_name,hue_value,hue_order,P5_index_seq,P7_index_seq,barcode_seq,N_total,N_CC,N_CCA,N_CCA+CC,CCA+CC_percent_total,percent_CCA,N_after_trim,N_UMI_observed,N_UMI_expected,percent_seqs_after_UMI_trim,percent_UMI_obs-vs-exp,N_after_downsample
0,A70,A70,1,2023-06-06/P12_R1.fastq.bz2,2023-06-06/P12_R2.fastq.bz2,D503,D704,l1Sp,human,RT-comp,Polymerase-sample,Maxima_S1,1,AGGATAGG,GAGATTCC,GGCTGCCATGCGACTA,3641249,363080,3247507,3610587,99.157926,89.944017,3565300,503559,523704.222531,97.914205,96.153321,2000000
1,A71,A71,1,2023-06-06/P12_R1.fastq.bz2,2023-06-06/P12_R2.fastq.bz2,D503,D704,l2Sp,human,RT-comp,Polymerase-sample,Maxima_S2,3,AGGATAGG,GAGATTCC,GGCTGCCATGCTGTCACG,3438157,229039,3179334,3408373,99.133722,93.280108,3365403,498815,523433.263275,97.883924,95.296771,2000000
2,A72,A72,1,2023-06-06/P12_R1.fastq.bz2,2023-06-06/P12_R2.fastq.bz2,D503,D704,l3Sp,human,RT-comp,Polymerase-sample,Maxima_S3,5,AGGATAGG,GAGATTCC,GGCTGCCATGCTGCGA,4065779,297561,3734550,4032111,99.171918,92.620218,3979477,507719,524023.050517,97.877356,96.888677,2000000


In [7]:
### Align reads to database of reference tRNAs ###
align_obj = SWIPE_align(dir_dict, tRNA_database_masked, sample_df, SWIPE_score_mat2, \
                        gap_penalty=6, extension_penalty=3, min_score_align=15, \
                        common_seqs=common_seqs, overwrite_dir=True)
sample_df = align_obj.run_parallel(n_jobs=8, overwrite=True, load_previous=False)
sample_df.head(3)

Using common sequences to prevent duplicated alignment.
Running Swipe on:  A76  A73  A74  A71  A72  A75  A70  A70_T  A71_T  A75_T  A72_T  A76_T  A74_T  common-seqs  A73_T

,sample_name_unique,sample_name,replicate,fastq_mate1_filename,fastq_mate2_filename,P5_index,P7_index,barcode,species,plot_group,hue_name,hue_value,hue_order,P5_index_seq,P7_index_seq,barcode_seq,N_total,N_CC,N_CCA,N_CCA+CC,CCA+CC_percent_total,percent_CCA,N_after_trim,N_UMI_observed,N_UMI_expected,percent_seqs_after_UMI_trim,percent_UMI_obs-vs-exp,N_after_downsample,N_mapped,percent_single_annotation,percent_multiple_annotation,percent_multiple_codons,Mapping_percent
0,A70,A70,1,2023-06-06/P12_R1.fastq.bz2,2023-06-06/P12_R2.fastq.bz2,D503,D704,l1Sp,human,RT-comp,Polymerase-sample,Maxima_S1,1,AGGATAGG,GAGATTCC,GGCTGCCATGCGACTA,3641249,363080,3247507,3610587,99.157926,89.944017,3565300,503559,523704.222531,97.914205,96.153321,2000000,1904893.0,64.536276,35.463724,6.109215,95.24465
1,A71,A71,1,2023-06-06/P12_R1.fastq.bz2,2023-06-06/P12_R2.fastq.bz2,D503,D704,l2Sp,human,RT-comp,Polymerase-sample,Maxima_S2,3,AGGATAGG,GAGATTCC,GGCTGCCATGCTGTCACG,3438157,229039,3179334,3408373,99.133722,93.280108,3365403,498815,523433.263275,97.883924,95.296771,2000000,1839343.0,64.155788,35.844212,6.244404,91.96715
2,A72,A72,1,2023-06-06/P12_R1.fastq.bz2,2023-06-06/P12_R2.fastq.bz2,D503,D704,l3Sp,human,RT-comp,Polymerase-sample,Maxima_S3,5,AGGATAGG,GAGATTCC,GGCTGCCATGCTGCGA,4065779,297561,3734550,4032111,99.171918,92.620218,3979477,507719,524023.050517,97.877356,96.888677,2000000,1904997.0,64.601362,35.398638,6.004366,95.24985


In [8]:
### Collect alignment statistics ###
stats_obj = STATS_collection(dir_dict, tRNA_data, sample_df, common_seqs=common_seqs, \
                             overwrite_dir=True)
stats_df = stats_obj.run_parallel(n_jobs=12, load_previous=False)
stats_df.head(3)

Using common sequences...

,sample_name_unique,sample_name,replicate,barcode,species,tRNA_annotation,tRNA_annotation_len,unique_annotation,5p_cover,align_3p_nt,codon,anticodon,amino_acid,align_gap,fmax_score>0.9,count,UMIcount,UMI_percent_exp
0,A70,A70,1,l1Sp,human,Escherichia_coli_str_K_12_substr_MG1655_tRNA-e...,76,True,False,A,AAA,TTT,eColiLys,False,False,2,2,100.000095
1,A70,A70,1,l1Sp,human,Escherichia_coli_str_K_12_substr_MG1655_tRNA-e...,76,True,False,A,AAA,TTT,eColiLys,False,True,4,4,100.000286
2,A70,A70,1,l1Sp,human,Escherichia_coli_str_K_12_substr_MG1655_tRNA-e...,76,True,False,C,AAA,TTT,eColiLys,False,False,91,91,100.008583


In [9]:
### Generate standard tRNAseq data plots ###
plot_obj = TRNA_plot(dir_dict, sample_df, overwrite_dir=True, pull_default=False)

# Add polymerase column to sample_df
# to group plots according to this:
sample_df['polymerase'] = [v.split('_')[0] for v in sample_df['hue_value']]

In [10]:
# Ecoli control:
plot_obj.plot_Ecoli_ctr()

In [11]:
# Codon abundance barchart:
plot_obj.plot_abundance(plot_type='codon', plot_name='codon_abundance_grp', \
                        group=True, min_obs=500)

# Codon charge barchart:
plot_obj.plot_abundance(plot_type='codon', plot_name='codon_charge_grp', \
                        group=True, min_obs=500, charge_plot=True)


Now plotting sample/group:  RT-comp
Now plotting sample/group:  RT-comp

In [12]:
# Transcript abundance comparison Maxima vs. TGIRT:
plot_obj.plot_abundance_corr(sample_unique_pairs=[['A70', 'A71', 'A72', 'A73', \
                                                   'A74', 'A75', 'A76'], \
                                                  ['A70_T', 'A71_T', 'A72_T', 'A73_T', \
                                                   'A74_T', 'A75_T', 'A76_T']], \
                             plot_type='transcript', plot_name='tr_abundance_corr_samples', \
                             min_obs=500, charge_plot=False, log=True)

# Transcript abundance comparison Maxima vs. TGIRT:
plot_obj.plot_abundance_corr(sample_unique_pairs=[['A70', 'A71', 'A72', 'A73', \
                                                   'A74', 'A75', 'A76'], \
                                                  ['A70_T', 'A71_T', 'A72_T', 'A73_T', \
                                                   'A74_T', 'A75_T', 'A76_T']], \
                             plot_type='transcript', plot_name='tr_charge_corr_samples', \
                             min_obs=500, charge_plot=True, one2one_corr=True)


Now plotting sample pairs:  (A70 - A70_T)  (A71 - A71_T)  (A72 - A72_T)  (A73 - A73_T)  (A74 - A74_T)  (A75 - A75_T)  (A76 - A76_T)
Now plotting sample pairs:  (A70 - A70_T)  (A71 - A71_T)  (A72 - A72_T)  (A73 - A73_T)  (A74 - A74_T)  (A75 - A75_T)  (A76 - A76_T)

In [13]:
# Transcript abundance comparison Maxima vs. TGIRT.
# Average over all the samples in each RT group:
plot_obj.plot_abundance_corr(sample_pairs=[['Maxima'], \
                                           ['TGIRT']], \
                             sample_pairs_col='polymerase', \
                             plot_type='codon', plot_name='codon_abundance_corr_RT', \
                             min_obs=1000, charge_plot=False, log=True)

# Transcript abundance comparison Maxima vs. TGIRT:
plot_obj.plot_abundance_corr(sample_pairs=[['Maxima'], \
                                           ['TGIRT']], \
                             sample_pairs_col='polymerase', \
                             plot_type='codon', plot_name='codon_charge_corr_RT', \
                             min_obs=1000, charge_plot=True, one2one_corr=True)


Now plotting sample pairs:  (Maxima - TGIRT)
Now plotting sample pairs:  (Maxima - TGIRT)

In [14]:
### Coverage plots for cyto/mito transcripts:
plot_obj.plot_coverage(compartment='cyto', plot_type='behrens', y_norm=True, \
                       plot_name='cov_plot_cyto_behrens_norm', n_jobs=12)

plot_obj.plot_coverage(compartment='mito', plot_type='behrens', y_norm=True, \
                       plot_name='cov_plot_mito_behrens_norm', n_jobs=12)


Now collecting data for sample:  A70  A71  A72  A73  A74  A75  A76  A70_T  A72_T  A73_T  A74_T  A71_T  A76_T  A75_T
Now plotting sample:  A70  A71  A72  A73  A74  A75  A76  A70_T  A71_T  A72_T  A73_T  A74_T  A75_T  A76_T
Now collecting data for sample:  A70  A72  A71  A75  A76  A70_T  A71_T  A72_T  A73  A73_T  A74_T  A74  A75_T  A76_T
Now plotting sample:  A70  A71  A72  A73  A74  A75  A76  A70_T  A71_T  A72_T  A73_T  A74_T  A75_T  A76_T

In [15]:
# UMI logo:
plot_obj.plot_UMI_logo(n_jobs=8)


Now collecting data for sample:  A70  A76  A71  A72  A73  A74  A75  A70_T  A72_T  A71_T  A75_T  A73_T  A74_T  A76_T
Now plotting logo plot.

In [16]:
# Non-template nucleotides:
plot_obj.plot_non_temp(end='5p', plot_name='_5p-non-template_logo', \
                       seq_len_percentile=99, n_jobs=8)

plot_obj.plot_non_temp(end='3p', plot_name='_3p-non-template_logo', \
                       seq_len_percentile=99.9, n_jobs=8)

plot_obj.plot_non_temp(end='3p', plot_name='_3p-non-template_3p-cover_logo', \
                       seq_len_percentile=99.9, _3p_cover=True, n_jobs=8)


Now collecting data for sample:  A70  A72  A71  A73  A74  A75  A76  A70_T  A74_T  A71_T  A73_T  A76_T  A72_T  A75_T
Now plotting logo plot.
Now collecting data for sample:  A70  A76  A71  A72  A73  A74  A75  A70_T  A75_T  A72_T  A73_T  A74_T  A76_T  A71_T
Now plotting logo plot.
Now collecting data for sample:  A70  A71  A72  A73  A74  A75  A76  A70_T  A74_T  A76_T  A72_T  A71_T  A73_T  A75_T
Now plotting logo plot.

In [17]:
### Perform transcript mutation analysis ###
TM_obj = TM_analysis(dir_dict, sample_df, tRNA_database, pull_default=False, \
                     common_seqs=common_seqs, ignore_common_count=False, \
                     overwrite_dir=True)

Using common sequences...


In [18]:
# Find mutations and save them:
TM_obj.find_muts(n_jobs=12, unique_anno=True)
TM_obj.pickle_muts_write(pickle_name='saved_muts_unique-anno.pickle')

In [19]:
# Compare mutations/gaps/RT stops by polymerase used:
TM_obj.plot_transcript_mut_compare(plot_name='mut_matrix_comp_top10-max-diff', \
                                   no_plot_return=True, \
                                   min_count_show=1000, \
                                   data_type='mut', \
                                   sample_pairs=[['Maxima'], \
                                                 ['TGIRT']], \
                                   sample_pairs_col='polymerase', \
                                   freq_avg_weighted=False, \
                                   topN=10, topN_select='max_diff')

TM_obj.plot_transcript_mut_compare(plot_name='gap_matrix_comp_top10-max-diff', \
                                   no_plot_return=True, \
                                   min_count_show=1000, \
                                   data_type='gap', \
                                   sample_pairs=[['Maxima'], \
                                                 ['TGIRT']], \
                                   sample_pairs_col='polymerase', \
                                   freq_avg_weighted=False, \
                                   topN=10, topN_select='max_diff')

TM_obj.plot_transcript_mut_compare(plot_name='RTstops_matrix_comp_top10-max-diff', \
                                   no_plot_return=True, \
                                   min_count_show=1000, \
                                   data_type='RTstops', \
                                   sample_pairs=[['Maxima'], \
                                                 ['TGIRT']], \
                                   sample_pairs_col='polymerase', \
                                   freq_avg_weighted=False, \
                                   topN=10, topN_select='max_diff')

In [20]:
# Compare mutations/gaps/RT stops by polymerase used
# but only for mitochondrial tRNA transcripts:
TM_obj.plot_transcript_mut_compare(plot_name='mut_matrix_comp_top10-max-diff_mito', \
                                   no_plot_return=True, \
                                   min_count_show=1000, \
                                   data_type='mut', \
                                   mito=True, \
                                   sample_pairs=[['Maxima'], \
                                                 ['TGIRT']], \
                                   sample_pairs_col='polymerase', \
                                   freq_avg_weighted=False, \
                                   topN=10, topN_select='max_diff')

TM_obj.plot_transcript_mut_compare(plot_name='gap_matrix_comp_top10-max-diff_mito', \
                                   no_plot_return=True, \
                                   min_count_show=1000, \
                                   data_type='gap', \
                                   mito=True, \
                                   sample_pairs=[['Maxima'], \
                                                 ['TGIRT']], \
                                   sample_pairs_col='polymerase', \
                                   freq_avg_weighted=False, \
                                   topN=10, topN_select='max_diff')

TM_obj.plot_transcript_mut_compare(plot_name='RTstops_matrix_comp_top10-max-diff_mito', \
                                   no_plot_return=True, \
                                   min_count_show=1000, \
                                   data_type='RTstops', \
                                   mito=True, \
                                   sample_pairs=[['Maxima'], \
                                                 ['TGIRT']], \
                                   sample_pairs_col='polymerase', \
                                   freq_avg_weighted=False, \
                                   topN=10, topN_select='max_diff')